In [1]:
import requests
import os
import zipfile
import openpyxl
import sqlite3
import re
import csv
import math

In [2]:
#create dir
def create_dir(dir_name):
    os.mkdir(dir_name)

In [3]:
#download binary file
def download_file_binary(url, file_name, dir_name=None):
    req = requests.get(url)
    if dir_name != None:
        file_name = os.path.join(dir_name, file_name)
    binary_file = open(file_name, "wb")
    binary_file.write(req.content)
    binary_file.close()

In [4]:
#unzip file
def unzip_file(file_name, dir_name=None):
    if dir_name != None:
        file_name = os.path.join(dir_name, file_name)
    z = zipfile.ZipFile(file_name, "r")
    z.extractall(dir_name)
    z.close()

In [5]:
#get a list of csv files under a given dir
def get_list_csv_files(dir_name):
    list_files = os.listdir(dir_name)
    list_csv_files = []
    for f in list_files:
        if f.endswith('csv'):
            list_csv_files.append(f)
    #ignore the broken csv file
    list_csv_files.remove('FY2015_Percent_Change_in_Medicare_Payments.csv')
    return list_csv_files

In [6]:
#transform a given name to be acceptable for sqlLight
#t_c_type: 't'- table, 'c' - column
def transform_table_and_column_name(name, t_c_type):
    name = name.lower()
    name = name.replace('%', 'pct')
    name = name.replace('/','_')
    
    regex = re.compile('[\s-]')
    name = regex.sub('_', name)
    
    regex = re.compile('^[a-z]')
    if regex.findall(name) == []:
        if t_c_type == 't':
            name = 't_' + name
        else:
            name = 'c_' + name
    return name

In [7]:
#get a table name from a csv file name or an excel sheet name
def get_table_name(file_name):
    if file_name.endswith('.csv'):
        table_name = file_name.split('.csv')[0]
    else:
        table_name = file_name
    #replace some illegal characters
    table_name = transform_table_and_column_name(table_name, 't')
    return table_name

In [8]:
#get a list of table columns from a csv file
def get_csv_columns(file_name, dir_name=None):
    file = os.path.join(dir_name, file_name)
    columns = []
    with open(file, 'rt', encoding='cp1252') as f:
        reader = csv.reader(f)
        for i, rows in enumerate(reader):
            if i == 0:
                for row in rows:
                    #replace some illegal characters, and specify the type for columns
                    columns.append(transform_table_and_column_name(row, 'c')+' text')
    return columns

In [9]:
#get a lit of fact data from a csv file
def read_data_from_csv_file(file_name, dir_name=None):
    file = os.path.join(dir_name, file_name)    
    data = []
    with open(file, 'rt', encoding='cp1252') as f:
        reader = csv.reader(f)
        for i, rows in enumerate(reader):
            if i > 0:
                data.append(rows)
    return data

In [10]:
#generate a list of table columns from a list of headers in a excel sheet
def get_excel_sheet_columns(headers=[]):
    columns = []
    for header in headers:
        columns.append(transform_table_and_column_name(header, 'c')+' text')
    return columns

In [11]:
#create a table and load data into it using the info read from a csv file or a worksheet
def load_file_to_database(cursor, table_name, columns=[], data=[]):
    #if the table exits, drop the table
    cursor.execute('DROP TABLE IF EXISTS {tn}'.format(tn=table_name))
    #create table
    cursor.execute('CREATE TABLE {tn} ({cn})'.format(tn=table_name,cn=', '.join(columns)))
    #load data into table
    for row in data:
        if row != [' ']:
            cursor.execute('INSERT INTO {tn} VALUES{d}'.format(tn=table_name,d=tuple(row)))    

In [12]:
#read header and fact data from an excel sheet
def read_excel_sheet(sheet):
    i = 1
    j = 1
    list_sheet = []
    while sheet.cell(row = i, column = j).value != None:
        list_row = []
        while sheet.cell(row = i, column = j).value != None:
            list_row.append(sheet.cell(row = i, column = j).value)
            j += 1
        else:
            j = 1
        list_sheet.append(list_row)
        i += 1
    else:
        return list_sheet

In [13]:
#create a worksheet
def create_worksheet(workbook, sheet_name, list_header_name=[], data=[]):
    worksheet = workbook.create_sheet(sheet_name)
    len_column = len(list_header_name)
    len_row = len(data)
    for j in range(len_column):
        worksheet.cell(row=1, column=j+1, value=list_header_name[j])
    for i in range(len_row):
        for j in range(len_column):
            worksheet.cell(row=2+i, column=j+1, value=data[i][j])

In [14]:
#the stdev class will be used to define an aggregate function in sqllight
class stdev:
    def __init__(self):
        self.mean = 0.0
        self.sample_var = 0.0
        self.count = 1

    def step(self, value):
        if value is None:
            return
        temp_mean = self.mean
        self.mean += (value - temp_mean) / self.count
        self.sample_var += (value - temp_mean) * (value - self.mean)
        self.count += 1

    def finalize(self):
        if self.count < 3:
            return None
        return math.sqrt(self.sample_var / (self.count-2))

define variables for sqlite

In [15]:
#create a sqlite connection
conn = sqlite3.connect('medicare_hospital_compare.db')
#create 'stdev' aggreage function
conn.create_aggregate("stdev", 1, stdev)
#create a cursor to execute sql script
cursor = conn.cursor()

download zip file and unzip

In [16]:
#define variables 
gov_zip_file_url = "https://data.medicare.gov/views/bg9k-emty/files/0a9879e0-3312-4719-a1db-39fd114890f1?content_type=application%2Fzip%3B%20charset%3Dbinary&filename=Hospital_Revised_Flatfiles.zip"
staging_dir_name = "staging"
gov_zip_file_name = "Hospital_Revised_Flatfiles.zip"

In [17]:
#download zip file and unzip
create_dir(staging_dir_name)
download_file_binary(gov_zip_file_url, gov_zip_file_name, staging_dir_name)
unzip_file(gov_zip_file_name, staging_dir_name)

load csv files into database

In [18]:
#get the list of csv files
list_csv_files = get_list_csv_files(staging_dir_name)

In [20]:
#laod all csv files into database
for f in list_csv_files:
    print(f)
    table_name = get_table_name(f)
    columns = get_csv_columns(f, staging_dir_name)
    data = read_data_from_csv_file(f, staging_dir_name)
    load_file_to_database(cursor, table_name, columns, data)

Ambulatory Surgical Measures-Facility.csv
Ambulatory Surgical Measures-National.csv
Ambulatory Surgical Measures-State.csv
Complications - Hospital.csv
Complications - National.csv
Complications - State.csv
Footnote Crosswalk.csv
FY2015_Distribution_of_Net_Change_in_Base_Op_DRG_Payment_Amt.csv
FY2015_Net_Change_in_Base_Op_DRG_Payment_Amt.csv
FY2015_Value_Based_Incentive_Payment_Amount.csv
GLOBAL_April2017_09March2017.csv
HCAHPS - Hospital.csv
HCAHPS - National.csv
HCAHPS - State.csv
Healthcare Associated Infections - Hospital.csv
Healthcare Associated Infections - National.csv
Healthcare Associated Infections - State.csv
Hospital General Information.csv
HOSPITAL_QUARTERLY_HAC_DOMAIN_HOSPITAL.csv
HOSPITAL_QUARTERLY_IPFQR_MEASURES_HOSPITAL.csv
HOSPITAL_QUARTERLY_IPFQR_MEASURES_NATIONAL.csv
HOSPITAL_QUARTERLY_IPFQR_MEASURES_STATE.csv
HOSPITAL_QUARTERLY_MSPB_6_DECIMALS.csv
HOSPITAL_QUARTERLY_QUALITYMEASURE_PCH_HCAHPS_HOSPITAL.csv
HOSPITAL_QUARTERLY_QUALITYMEASURE_PCH_HCAHPS_NATIONAL.csv
HO

download the excel file and load it to database

In [21]:
#define variables
inhouse_excel_file_url = "http://kevincrook.com/utd/hospital_ranking_focus_states.xlsx"
inhouse_excel_file_name = "hospital_ranking_focus_states.xlsx"

In [22]:
#download the inhouse excel file
download_file_binary(inhouse_excel_file_url, inhouse_excel_file_name)

In [23]:
#read inhouse excel file book 
workbook = openpyxl.load_workbook(inhouse_excel_file_name)
list_sheet_names = workbook.get_sheet_names()

In [24]:
#read inhouse excel file sheets and load into database
for sheet_name in list_sheet_names:
    #read sheet
    list_sheet_contents = []
    sheet = workbook.get_sheet_by_name(sheet_name)
    list_sheet_contents = read_excel_sheet(sheet)    
    #load sheet into database    
    table_name = get_table_name(sheet_name)
    columns = get_excel_sheet_columns(list_sheet_contents[0])
    data = list_sheet_contents[1:]
    load_file_to_database(cursor, table_name, columns, data)

create the hospital ranking excel workbook

In [25]:
#create the hospital ranking workbook 
wb_ranking = openpyxl.Workbook()

In [26]:
#execute query for nationwide hospital ranking
cursor.execute('SELECT T.provider_id, LJT.hospital_name, LJT.city, LJT.state, LJT.county_name \
    FROM hospital_national_ranking T LEFT JOIN hospital_general_information LJT ON T.provider_id = LJT.provider_id \
    WHERE CAST(T.ranking AS INT) <= 100 ORDER BY T.ranking')
list_national_ranking = cursor.fetchall()

In [27]:
#save the result to excel sheet
sheet_name = 'Nationwide'
list_header_name = ['Provider ID', 'Hospital Name', 'City', 'State', 'County']
create_worksheet(wb_ranking, sheet_name, list_header_name, list_national_ranking)

In [28]:
#execute query for the list of focus state
cursor.execute('SELECT state_name, state_abbreviation FROM focus_states ORDER BY state_name')
list_focus_states = cursor.fetchall()

In [29]:
#execute query for hospital ranking in each focus state and save the results to excel sheets
for state in list_focus_states:
    state_name = state[0]
    state_abbr = state[1]
    list_state_ranking = []
    
    #execute the query
    cursor.execute('SELECT T.provider_id, LJT.hospital_name, LJT.city, LJT.state, LJT.county_name \
    FROM hospital_national_ranking T LEFT JOIN hospital_general_information LJT ON T.provider_id = LJT.provider_id \
    WHERE LJT.state == "{st}" ORDER BY T.ranking'.format(st=state_abbr))
    all_rows = cursor.fetchall()
    #only take the top 100 records
    list_state_ranking = all_rows[:100]
    
    #save the result to excel sheet    
    sheet_name = state_name
    create_worksheet(wb_ranking, sheet_name, list_header_name, list_state_ranking)

In [30]:
#save the hospital_ranking workbook
wb_ranking.remove_sheet(wb_ranking.get_sheet_by_name("Sheet"))
wb_ranking.save("hospital_ranking.xlsx")

create the measure statistics excel workbook

In [31]:
#create the measure statistics workbook 
wb_stat = openpyxl.Workbook()

In [32]:
#execute query for nationwide statistics. STDEV is a manually added aggreage function
cursor.execute('SELECT measure_id, measure_name, MIN(CAST(score AS FLOAT)),MAX(CAST(score AS FLOAT)), \
    ROUND(AVG(CAST(score AS FLOAT)),4), ROUND(STDEV(CAST(score AS FLOAT)),4) \
    FROM timely_and_effective_care___hospital WHERE score==score+0 \
    GROUP BY measure_id, measure_name ORDER BY measure_id')
list_nationawide_statistics = cursor.fetchall() 

In [33]:
#save the result to excel sheet 
sheet_name = 'Nationwide'
list_header_name = ['Measure ID', 'Measure Name', 'Minimum', 'Maximum', 'Average', 'Standard Deviation']
create_worksheet(wb_stat, sheet_name, list_header_name, list_nationawide_statistics)

In [34]:
#execute query for measure statistics in each focus state and save results to excel sheets
for state in list_focus_states:
    state_name = state[0]
    state_abbr = state[1]
    list_state_statistics = []    
    
    #'WHERE score==score+0' will exclude the records having non-numaric score'
    cursor.execute('SELECT measure_id, measure_name, MIN(CAST(score AS FLOAT)),MAX(CAST(score AS FLOAT)), \
        ROUND(AVG(CAST(score AS FLOAT)),4), ROUND(STDEV(CAST(score AS FLOAT)),4) \
        FROM timely_and_effective_care___hospital WHERE score==score+0 and state == "{st}"\
        GROUP BY measure_id, measure_name ORDER BY measure_id'.format(st=state_abbr))
    list_state_statistics = cursor.fetchall()
    
    #save the result to excel sheet
    sheet_name = state_name
    create_worksheet(wb_stat, sheet_name, list_header_name, list_state_statistics)

In [35]:
#save the measure statistics workbook
wb_stat.remove_sheet(wb_stat.get_sheet_by_name("Sheet"))
wb_stat.save("measure_statistics.xlsx")